In [1]:
%run base.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
x_next, states, u, dt, q_min, q_max ,parameters, _   = rig_dyn.forward_dynamics(gravity=9.81, floating_base_id=None, floating_base_bias_f=None, J_uv=None, coupled=False)
F_next_ = cs.Function('forward_dynamics_base', [states, u, parameters, dt, q_min, q_max], [x_next, u])
F_next_.save("ARM_model.casadi")

no floating_base found
no floating_base found
no floating_base found


In [3]:
q0 = [1.91318, 1.33339, 0.99245, 3.16073]
q0dot = [0.00751799, -0.0073626, 0.00281749, 0.0222385]
u0 = [0.0340454, -0.00441695, 0.0062739, -0.00505981]
state0 = cs.vertcat(q0, q0dot)

nextState , u = F_next_(state0, u0, alpha.sim_p, alpha.delta_t, alpha.joint_min, alpha.joint_max)
print(nextState)

[1.91372, 1.5, 0.992592, 3.16066, 0.0113435, 0, 0, 0]


In [4]:
# # c , cpp or matlab code generation for forward dynamics
# F_next_d.generate("F_next_.c")
# os.system(f"gcc -fPIC -shared F_next_.c -o libMnext.so")